# Online Surrogate Model Dashboard Demo

Note: Complete Server.ipynb notebook before moving forward with dashboard.

## Project description 

In this demo, we will launch a PVAccess server and use its process variables as the inputs and outputs of an online surrogate model. We will then create an application using sliders to control the input process variables, and a number of data views to capture the ouput variables.This is the second of two notebooks used for the demo. In this notebook, we set up the dashboard view of the model. The application will consist of: 
- Input control sliders
- Image view
- Value table
- Striptool

Each of these items will be referred to as "widgets". 

## Structure of the application folder

All pages that can be served using the command `bokeh serve {page} --args -p pva` are located in the `pages` subfolder. Widget items used to design an application are found in the `widgets` subfolder. All widgets make use of monitors, which format the  data appropriately for widget use. The monitors use a defined controller to put and get process variables from the PVAccess server.

```
app
├── __init__.py
├── controllers.py
├── monitors.py
├── pages
│   ├── __init__.py
│   ├── controls.py
│   ├── dashboard.py
│   ├── image_viewer.py
│   └── striptool.py
└── widgets
    ├── __init__.py
    ├── plots.py
    ├── sliders.py
    └── tables.py
```

## Note on running- requires active server
In addition to the launching the server using Server.ipynb, the sever can also be started from an additional terminal using the conda environment and the command `python bin/cli.py serve start-server pva`.

# Application design
The application was designed in the spirit of a model-view-controller framework, with a specific departure: the controller is responsible for both serving and fetching information from the server. Views are updated based on changes in the controllers report of the output states of the output variables.

<img src="online_model/files/dashboard_app_diagram.png">

In [ ]:
# MAKE SURE THAT THE REPOSITORY ROOT IS IN THE PYTHONPATH
import sys
import os

module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from bokeh.io import output_notebook, show
from bokeh.models.widgets import Select
from bokeh import palettes
from bokeh.layouts import column, row, Spacer

# load bokeh
output_notebook()

In [ ]:
# Import the controller
from online_model.app.controllers import Controller

# Import the widgets
from online_model.app.widgets.sliders import build_sliders
from online_model.app.widgets.plots import ImagePlot, Striptool
from online_model.app.widgets.tables import ValueTable

# Import the model specific information for building the view
from online_model import PREFIX, SIM_PVDB, CMD_PVDB, EXCLUDE_SLIDERS

# need surrogate model image processing methods
from online_model.model.MySurrogateModel import MySurrogateModel

# Set up the controller
The Controller class has `get` and `put` methods that are handled using the appropriate protocol. 

In [ ]:
PROTOCOL = "pva"

# create controller
controller = Controller(PROTOCOL)

# Build sliders
Sliders are build for any input process variable that is not in the EXCLUDE_SLIDERS list (these are the input extents right now),

In [ ]:
# build sliders for the command process variable database
# TEMPORARILY EXCLUDE THE EXTENTS
sliders_to_render = {}
for var, value in CMD_PVDB.items():
    if "in_" not in EXCLUDE_SLIDERS:
        sliders_to_render[var] = value

sliders = build_sliders(sliders_to_render, controller)

# Image Viewer

The image viewer has two associated items, the selection tool and the plot. The selection tool is used for selecting the axes to examine (are we looking at x vs. y, x vs. z, etc.) and the plot item actually displays the image. 

## Callbacks
For both the selection tool and the plot, we have to define the appropriate callbacks for updating data. The `image_select` callback is used for toggling the variable to display. The `image_update_callback` syncs the image presented with the output variables collected from the server.


In [ ]:
# Create custom palette with low values set to white
pal = list(palettes.viridis(244))  # 256 - 12 (set lowest 5% to white)
pal = ["#FFFFFF"] * 12 + pal
pal = tuple(pal)

# create plot
# need to pass MySurrogateModel class into the plot because need access to the prepare_image_from_pv method
image_plot = ImagePlot(SIM_PVDB, controller, MySurrogateModel)
image_plot.build_plot(pal)

# track current_pv globally
current_image_pv = image_plot.current_pv

# set up image toggle
image_select = Select(
    title="Image PV",
    value=current_image_pv,
    options=list(image_plot.pv_monitors.keys()),
)

# callback to update the image variables
def on_image_selection(attrname, old, new):
    """
    Callback function for dropdown selection that updates the global current variable.
    """
    global current_image_pv
    current_image_pv = new

#assign the callback to the image selection
image_select.on_change("value", on_image_selection)

# Set up image update callback
def image_update_callback():
    """
    Calls plot controller update with the current global process variable
    """
    global current_image_pv
    image_plot.update(current_image_pv)

# Striptool
The striptool requires the same update and selection callbacks as the image_viewer.

In [ ]:
# Set up the striptool
striptool = Striptool(SIM_PVDB, controller)
striptool.build_plot()

# set up global pv
current_striptool_pv = striptool.current_pv

# create a selection tool so we can switch between output variables
striptool_select = Select(
    title="PV to Plot:",
    value=current_striptool_pv,
    options=list(striptool.pv_monitors.keys()),
)

# create a selection callback
def striptool_select_callback(attr, old, new):
    global current_striptool_pv
    current_striptool_pv = new
    
# striptool data update callback
def striptool_update_callback():
    """
    Calls plot controller update with the current global process variable
    and updates the value table.
    """
    global current_striptool_pv
    striptool.update(current_striptool_pv)
    value_table.update()

# assign the selection callback to the striptool
striptool_select.on_change("value", striptool_select_callback)

# Value table
The value table requires an update callback.

In [ ]:
# add table
value_table = ValueTable(SIM_PVDB, controller)

# Set up callback to update the table
def table_update_callback():
    """
    Updates the value table.
    """
    value_table.update()

# Set up our application layout
The `render_app` function will start the dashboard within the notebook. This uses bokeh's row and column layout items to compose the view. We also add our update callbacks to the document. Finally, `show(render_app)` will embed the view.

# NOTE:
Depending on the order in which you opened your server and dashboard notebooks, you may run into a bokeh error, ``ERROR:bokeh.server.views.ws:Refusing websocket connection from Origin 'http://localhost:{SOME PORT}'``. This error is because coss site connections to the Bokeh server websocket aren't allowed by default. Running the command that follows the `render_app` definition, with the appropriate port from the error, will allow the app to be rendered.

In addtion you attempt to run the following code multiple times, without stopping the kernel, you will run into the following error: ``RuntimeError: Models must be owned by only a single document``. Restarting the kernel will remove this error.

In [ ]:
def render_app(doc):
    """
    Function for rendering the application within the embedded bokeh server.
    """
    doc.title = "Online Surrogate Model Image Viewer"
    doc.add_root( column(
        row(column(sliders, width=350), Spacer(width=50),  column(value_table.table, height=300)),  # add sliders
        row(column(image_select, image_plot.p), column(striptool_select, striptool.p))
    ))
    doc.add_periodic_callback(image_update_callback, 250)
    doc.add_periodic_callback(striptool_update_callback, 250)
    doc.add_periodic_callback(table_update_callback, 250)

    
show(render_app)

In [ ]:
# ALLOW CROSS SITE SCRIPTING
os.environ["BOKEH_ALLOW_WS_ORIGIN"] = "localhost:8889"